In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from paralleltomo import paralleltomo
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
path = "/Users/ninapeuker/Desktop/General_Engineering/6th semester 2023/02526 Mathematical Modeling/02526 Code/testImage.npy"

In [ ]:
im = np.load(path)
print(im.shape)

plt.imshow(im)
plt.show()

In [27]:
N=8
theta = np.matrix([45.0000,   67.5000,   90.0000,  112.5000,  135.0000,  157.5000,  180.0000,  202.5000,  225.0000,  247.5000,  270.0000, 292.5000,  315.0000])
[A,theta,p,d] = paralleltomo(N,theta,11)

# print([A,theta,p,d])
print(f"A = {A}")
print(np.linalg.matrix_rank(A),A.shape[0])
print()

N=200
theta =np.matrix(np.linspace(0,179,179))
p = 250
[A,theta,p,d] = paralleltomo(N,theta,p)
print(f"A = {A}")
# print([A,theta,p,d])

A = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.13137085 0.         0.        ]
 [0.         0.         0.         ... 0.         0.84852814 0.56568542]
 [0.         0.         0.         ... 0.         0.         0.        ]]
63 143

A = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
# Problem 1
N = 50
[A,theta,p,d] = paralleltomo(N)
print(np.linalg.matrix_rank(A),A.shape)


print(f"A = {A}")

/Users/ninapeuker/Desktop/General_Engineering/6th semester 2023/02526 Mathematical Modeling/02526 Code/MathModellingDTU/Exam Project/paralleltomo.py:102: RuntimeWarning: invalid value encountered in true_divide
  tx = (x - x0theta[j,0])/a


2500 (12780, 2500)
A = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
